# Reproducibility and ML-Flow

# What is Reproducibility?
`McDermott M, Wang S, Marinsek N, Ranganath R, Ghassemi M, Foschini L. Reproducibility in Machine Learning for Health. arXiv preprint arXiv:1907.01463. 2019 Jul 2.`

- Technical replicability
- Statistical replicability
- Conceptual replicability

Technical replicability refers to the ability to yield the precise results reported in the paper eg. code and dataset version. 

Statistical replicability refers to the ability of a result to hold under re-sampled conditions. New data that belongs to the same distribution.

Conceptual replicability is replicability under conditions that mach the conceptual description of the purported effect. 

"All three of these replicability criteria are central for full reproducibility: without technical replicability, one’s result cannot be demonstrated. Without statistical replicability, one’s result will not reproduce under increased sampling and the presence of real-world variance. And lastly, without conceptual replicability, one’s result does not depend on the desired properties of the data, but instead depends on potentially unobserved aspects of the data generation mechanism that, critically, will not reproduce when deployed in practice"

# Challenges in Healthcare

- Privacy concerns
- Lack of transparency in methods and code sharing
- Lack of multi-institution datasets
- Lack of standards in reporting on models

- Only 51% of the ML4H papers examined used public datasets, as compared to over 90% of both CV and NLP papers, and approximately 77% of general ML papers.
- Only approximately 13% of the ML4H papers analyzed released their code publicly, compared to approximately 37% in CV and 50% in NLP.
- Generalizing over changing care practices or data formats is challenging
- Performance degrades over time as care patterns evolve

![Figure 1](img/fig1.png)
`McDermott M et al`

# What is ML-Flow?

#### "MLflow is an open source platform to manage the ML lifecycle, including experimentation, reproducibility and deployment."

https://mlflow.org

# Integrations
![Figure 2](img/fig2.png)

# Example
https://mlflow.org/docs/latest/tutorial.html

In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
warnings.filterwarnings("ignore")
np.random.seed(40)

In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

# Load and split data

In [4]:
wine_path = "example/wine-quality.csv"
data = pd.read_csv(wine_path)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
if not (len(sys.argv) > 2 and sys.argv[2][0] == '/'):
    argv = sys.argv
else:
    argv = []
alpha = float(argv[1]) if len(argv) > 1 else 0.5
l1_ratio = float(argv[2]) if len(argv) > 2 else 0.5

# Train and log data

In [7]:
def run_once(alpha, l1_ratio):
    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [8]:
from itertools import product
alpha_l1 = product(np.arange(0,1,0.25), np.arange(0,1,0.25))

for params in alpha_l1:
    run_once(*params)

Elasticnet model (alpha=0.000000, l1_ratio=0.000000):
  RMSE: 0.7424162938559538
  MAE: 0.5775168907128364
  R2: 0.2881067715835015
Elasticnet model (alpha=0.000000, l1_ratio=0.250000):
  RMSE: 0.7424162938559538
  MAE: 0.5775168907128364
  R2: 0.2881067715835015
Elasticnet model (alpha=0.000000, l1_ratio=0.500000):
  RMSE: 0.7424162938559538
  MAE: 0.5775168907128364
  R2: 0.2881067715835015
Elasticnet model (alpha=0.000000, l1_ratio=0.750000):
  RMSE: 0.7424162938559538
  MAE: 0.5775168907128364
  R2: 0.2881067715835015
Elasticnet model (alpha=0.250000, l1_ratio=0.000000):
  RMSE: 0.7809291909864666
  MAE: 0.611562670306637
  R2: 0.2123320286653423
Elasticnet model (alpha=0.250000, l1_ratio=0.250000):
  RMSE: 0.7894718330025422
  MAE: 0.6172327290956597
  R2: 0.19500505762673592
Elasticnet model (alpha=0.250000, l1_ratio=0.500000):
  RMSE: 0.7975067375423913
  MAE: 0.6207309553456548
  R2: 0.17853588976514378
Elasticnet model (alpha=0.250000, l1_ratio=0.750000):
  RMSE: 0.80505359500

# Viewing your results

In [ ]:
!mlflow ui

[2019-09-04 19:22:19 -0600] [14029] [INFO] Starting gunicorn 19.9.0
[2019-09-04 19:22:19 -0600] [14029] [INFO] Listening at: http://127.0.0.1:5000 (14029)
[2019-09-04 19:22:19 -0600] [14029] [INFO] Using worker: sync
[2019-09-04 19:22:19 -0600] [14032] [INFO] Booting worker with pid: 14032
[2019-09-05 10:05:40 -0600] [14029] [CRITICAL] WORKER TIMEOUT (pid:14032)
[2019-09-05 10:05:40 -0600] [14032] [INFO] Worker exiting (pid: 14032)
[2019-09-05 10:05:40 -0600] [14700] [INFO] Booting worker with pid: 14700


# Packaging Your Model

Define two files in the root directory of your project

- conda.yaml
- MLproject


### MLproject
```yaml
name: tutorial

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"
    
```

### conda.yaml
```yaml
name: tutorial
channels:
  - defaults
dependencies:
  - python=3.6
  - scikit-learn=0.19.1
  - pip:
    - mlflow>=1.0

```

In [12]:
!mlflow run example -P alpha=0.42

2019/08/29 17:44:52 INFO mlflow.projects: === Creating conda environment mlflow-266307e4c5272b86e4ea7b8d86fed73b47d05dd6 ===
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.4
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



_libgcc_mutex-0.1    | 3 KB      | ##################################### | 100% 
intel-openmp-2019.4  | 876 KB    | ##################################### | 100% 
python-3.6.9         | 34.4 MB   | ##################################### | 100% 
ca-certificates-2019 | 134 KB    | ##################################### | 100% 
libgcc-ng-9.1.0      | 8.1 MB    | ##################################### | 100% 
numpy-1.15.4         | 35 KB     | ##################################### | 100% 
numpy-base-1.15.4    | 4.2 MB    | ##################################### | 100% 
mkl-2018.0.3         | 198.7 MB  | ##################################### | 100% 
setuptools-41.0.1 

     |████████████████████████████████| 5.9MB 752kB/s eta 0:00:01
     |████████████████████████████████| 10.5MB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 1.1MB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8e/c7/70bd352e8a561a9b6d1cde9aa313b9d7c871b0c94c3821f44c01f3187e1d/GitPython-3.0.2-py3-none-any.whl
     |████████████████████████████████| 266kB 1.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/8c/da/b8dd8deb741bff556db53902d4706774c8e1e67265f69528c14c003644e6/gunicorn-19.9.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ef/53/900f7d2a54557c6a37886585a91336520e5539e3ae2423ff1102daf4f3a7/sqlparse-0.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb

  Using cached https://files.pythonhosted.org/packages/1d/e7/fd8b501e7a6dfe492a433deb7b9d833d39ca74916fa8bc63dd1a4947a671/Jinja2-2.10.1-py2.py3-none-any.whl
     |████████████████████████████████| 337kB 747kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/29/19/44753eab1fdb50770ac69605527e8859468f3c0fd7dc5a76dd9c4dbd7906/websocket_client-0.56.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c6/d3/201fc3abe391bbae6606e6f1d598c15d367033332bd54352b12f35513717/python_editor-1.0.4-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/2c/cd551d81dbe15200be1cf41cd03869a46fe7226e7450af7a6545bfc474c9/idna-2.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/55/d2/866d45e3a121ee15a1dc013824d58072fd5c7799c9c34d01378eb262ca8f/smmap2-2.0.5-py2.py3

# Questions?